## CS336 - Lec1: Overview, tokenization

## Lec1: 
### 课程介绍

#### 课程收获
There are three types of knowledge:
    
1. *Mechanics: how things work (what a Transformer is, how model parallelism leverages GPUs)
2. *Mindset: squeezing the most out of the hardware, taking scale seriously (scaling laws)
3. Intuitions: which data and modeling decisions yield good accuracy

#### 目标
The bitter lesson
- accuracy = efficiency x resources
- Framing: what is the best model one can build given a certain compute and data budget?
In other words, maximize efficiency!

#### Current Landscape
1. Pre-neural(before 2010s)
    - Language model to measure the entropy of English [Shannon 1950]
    - Lots of work on n-gram language models (for machine translation, speech recognition) [Brants+ 2007]
2. Neural ingredients (2010s)    
    - First neural language model [Bengio+ 2003]
    - Sequence-to-sequence modeling (for machine translation) [Sutskever+ 2014]
    - Adam optimizer [Kingma+ 2014]
    - Attention mechanism (for machine translation) [Bahdanau+ 2014]
    - Transformer architecture (for machine translation) [Vaswani+ 2017]
    - Mixture of experts [Shazeer+ 2017]
    - Model parallelism [Huang+ 2018][Rajbhandari+ 2019][Shoeybi+ 2019]
3. Early foundation models (late 2010s)
    - ELMo: pretraining with LSTMs, fine-tuning helps tasks [Peters+ 2018]
    - BERT: pretraining with Transformer, fine-tuning helps tasks [Devlin+ 2018]
    - Google's T5 (11B): cast everything as text-to-text [Raffel+ 2019]
4. Embracing scaling, more closed
    - OpenAI's GPT-2 (1.5B): fluent text, first signs of zero-shot, staged release [Radford+ 2019]
    - Scaling laws: provide hope / predictability for scaling [Kaplan+ 2020]
    - OpenAI's GPT-3 (175B): in-context learning, closed [Brown+ 2020]
    - Google's PaLM (540B): massive scale, undertrained [Chowdhery+ 2022]
    - DeepMind's Chinchilla (70B): compute-optimal scaling laws [Hoffmann+ 2022]
5. Open models    
    - EleutherAI's open datasets (The Pile) and models (GPT-J) [Gao+ 2020][Wang+ 2021]
    - Meta's OPT (175B): GPT-3 replication, lots of hardware issues [Zhang+ 2022]
    - Hugging Face / BigScience's BLOOM: focused on data sourcing [Workshop+ 2022]
    - Meta's Llama models [Touvron+ 2023][Touvron+ 2023][Grattafiori+ 2024]
    - Alibaba's Qwen models [Qwen+ 2024]
    - DeepSeek's models [DeepSeek-AI+ 2024][DeepSeek-AI+ 2024][DeepSeek-AI+ 2024]
    - AI2's OLMo 2 [Groeneveld+ 2024][OLMo+ 2024]
6. Today's frontier models
    - [OpenAI's o3](https://openai.com/index/openai-o3-mini/)
    - [Anthropic's Claude Sonnet 3.7](https://www.anthropic.com/news/claude-3-7-sonnet)
    - [xAI's Grok 3](https://x.ai/news/grok-3)
    - [Google's Gemini 2.5](https://blog.google/technology/google-deepmind/gemini-model-thinking-updates-march-2025/)
    - [Meta's Llama 3.3](https://ai.meta.com/blog/meta-llama-3/)
    - [DeepSeek's r1]
    - [Alibaba's Qwen 2.5 Max](https://qwenlm.github.io/blog/qwen2.5-max/)
    - [Tencent's Hunyuan-T1](https://tencent.github.io/llm.hunyuan.T1/README_EN.html)

#### 适用人群
- Why you should take this course 
    - You have an obsessive need to understand how things work.
    - You want to build up your research engineering muscles.
- Why you should not take this course
    - You actually want to get research done this quarter.(Talk to your advisor.)
    - You are interested in learning about the hottest new techniques in AI (e.g., multimodality, RAG, etc.).(You should take a seminar class for that.)
    - You want to get good results on your own application domain.(You should just prompt or fine-tune an existing model.)

#### 课程组成
- It's all about efficiency. Efficiency drives design decisions.
    1. Basics: Tokenization, Architecture, Loss Function, Optimizer, Learning rate
    2. Systems: Kernels, Parallelism, Quantilization, Activation checkpoint, CPU offloading, Inference
    3. Scaling Laws: Scaling sequence, Model complexity, Loss metric, Parametric form
    4. Data: Evaluation, Curation, Transformation, Filtering, Deduplication, Mixing
    5. Alignment: Supervised fine-tuning, Reinforment Learning, Preference data, Synthetic data, Verifiers


##### 基础 Basics
- 目标: 得到能够运行的基础版本流程 
- 组成部分: 分词, 模型架构, 训练
1. 分词 Tokenization
    - Tokenizers convert between strings and sequences of integers (tokens)
    - Intuition: break up string into popular segments
    - This course: Byte-Pair Encoding (BPE) tokenizer
2. 模型架构
    - Starting point: original Transformer 
    - 变体：
        - Activation functions: ReLU, SwiGLU
        - Positional encodings: sinusoidal, RoPE
        - Normalization: LayerNorm, RMSNorm
        - Placement of normalization: pre-norm versus post-norm
        - MLP: dense, mixture of experts
        - Attention: full, sliding window, linear
        - Lower-dimensional attention: group-query attention (GQA), multi-head latent attention (MLA)
        - State-space models: Hyena
3. 训练
    - Optimizer (e.g., AdamW, Muon, SOAP)
    - Learning rate schedule (e.g., cosine, WSD)
    - Batch size (e..g, critical batch size)
    - Regularization (e.g., dropout, weight decay)
    - Hyperparameters (number of heads, hidden dimension): grid search

##### 系统 Systems
- 目标: 压榨硬件性能
- 组成部分: 计算核 kernels, 并行 parallelism, 推理 inference

...
1. Kernels 
    - Trick: organize computation to maximize utilization of GPUs by minimizing data movement
    - Write kernels in CUDA/Triton/CUTLASS/ThunderKittens
2. Parallelism
    - What if we have multiple GPUs (8 A100s)?
    - Data movement between GPUs is even slower, but same 'minimize data movement' principle holds
    - Use collective operations (e.g., gather, reduce, all-reduce)
    - Shard (parameters, activations, gradients, optimizer states) across GPUs
    - How to split computation: {data,tensor,pipeline,sequence} parallelism
3. Inference
    - Goal: generate tokens given a prompt (needed to actually use models!)
    - Inference is also needed for reinforcement learning, test-time compute, evaluation
    - Globally, inference compute (every use) exceeds training compute (one-time cost)
    - Two phases: prefill and decode
        - Prefill (similar to training): tokens are given, can process all at once (compute-bound)
        - Decode: need to generate one token at a time (memory-bound)
        - Methods to speed up decoding:
            - Use cheaper model (via model pruning, quantization, distillation)
            - Speculative decoding: use a cheaper "draft" model to generate multiple tokens, then use the full model to score in parallel (exact decoding!)
            - Systems optimizations: KV caching, batching

##### Scaling Laws
- 目标: do experiments at small scale, predict hyperparameters/loss at large scale
- 问题: given a FLOPs budget (C), use a bigger model (N) or train on more tokens (D)?

##### 数据 Data
- Question: What capabilities do we want the model to have? Multilingual? Code? Math?

1. Evaluation
    - Perplexity: textbook evaluation for language models
    - Standardized testing (e.g., MMLU, HellaSwag, GSM8K)
    - Instruction following (e.g., AlpacaEval, IFEval, WildBench)
    - Scaling test-time compute: chain-of-thought, ensembling
    - LM-as-a-judge: evaluate generative tasks
    - Full system: RAG, agents
2. Data curation
    - Data does not just fall from the sky.
    - Sources: webpages crawled from the Internet, books, arXiv papers, GitHub code, etc.
    - Appeal to fair use to train on copyright data?
    - Might have to license data (e.g., Google with Reddit data)
    - Formats: HTML, PDF, directories (not text!)
3. Data processing
    - Transformation: convert HTML/PDF to text (preserve content, some structure, rewriting)
    - Filtering: keep high quality data, remove harmful content (via classifiers)
    - Deduplication: save compute, avoid memorization; use Bloom filters or MinHash

##### 对齐 Alignment
- So far, a base model is raw potential, very good at completing the next token.
- Alignment makes the model actually useful.
- Goals of alignment:
    - Get the language model to follow instructions
    - Tune the style (format, length, tone, etc.)
    - Incorporate safety (e.g., refusals to answer harmful questions)
- Two phases:
    - `supervised_finetuning()`
    - `learning_from_feedback()`

1. Supervised finetuning (SFT)
    - Instruction data: (prompt, response) pairs
    - 示例：
        ```python
        sft_data: list[ChatExample] = [
            ChatExample(
                turns=[
                    Turn(role="system", content="You are a helpful assistant."),
                    Turn(role="user", content="What is 1 + 1?"),
                    Turn(role="assistant", content="The answer is 2."),
                ],
            ),
        ]
        ```
    - Data often involves human annotation.
    - Intuition: base model already has the skills, just need few examples to surface them.
    - Supervised learning: fine-tune model to maximize p(response | prompt).
2. Now we have a preliminary instruction following model.Let's make it better without expensive annotation.
3. 偏好数据 Preference data
    -  Data: generate multiple responses using model (e.g., [A, B]) to a given prompt.
    - User provides preferences (e.g., A < B or A > B).
    - 示例：
        ```python
        preference_data: list[PreferenceExample] = [
            PreferenceExample(
                history=[
                    Turn(role="system", content="You are a helpful assistant."),
                    Turn(role="user", content="What is the best way to train a language model?"),
                ],
                response_a="You should use a large dataset and train for a long time.",
                response_b="You should use a small dataset and train for a short time.",
                chosen="a",
            )
        ]
        ```
4. Verifiers
    - Formal verifiers (e.g., for code, math)
    - Learned verifiers: train against an LM-as-a-judge
5. Algorithms
    - Proximal Policy Optimization (PPO) from reinforcement learning
    - Direct Policy Optimization (DPO): for preference data, simpler
    - Group Relative Preference Optimization (GRPO): remove value function

### 分词器 Tokenizers

In [ ]:
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py#L23
GPT2_TOKENIZER_REGEX = \
    r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

#### 分词总结
参考 Andrej Karpathy[视频](https://www.youtube.com/watch?v=zduSFxRajkE)
- 分词器将`strings`转换为`tokens`/`indices`/`integer`
- 方式：基于Character、基于Byte、基于Word的分词
- 至今，Byte Pair Encoding BPE分词器效果最佳

Intro：
- 示例：原文本 string = "Hello, 🌍! 你好!" <=> 分词后文本表示为 indices = [15496, 11, 995, 0]
- 分词器可以实现encode和decode的功能的类
- 其中的词表大小是可能的tokens的数量
- 尝试用[GPT2的分词器](https://tiktokenizer.vercel.app/?encoder=gpt2)

1. 基于Character的分词
- Unicode的string
```python
    assert ord("a") == 97
    assert ord("🌍") == 127757

    assert chr(97) == "a"
    assert chr(127757) == "🌍"
```
- 问题：词表很大，且大多数词使用很少

2. 基于Byte的分词
- Unicode的string可以表示为一串字节（0-255），最常见的Unicode编码是UTF-8
- 问题：原句子的序列长度会非常长

3. 基于Word的分词
- 可以直接用regex来分词，但是问题和Char的分词一样且需要没见过的词的token UNK

4. BPE分词
- 想法: 用原始文本训练分词器，自动确定词汇表。常见的字符序列表示为单个token，罕见的字符序列表示为多个token
- 过程：
    1. 开始将每个byte看作一个token/word-based分词
    2. 迭代，每次合并最常出现的相邻的token pair，作为新的token值

In [ ]:
from collections import defaultdict

string = "the cat in the hat"

class BPETokenizerParams:
    """All you need to specify a BPETokenizer."""
    vocab: dict[int, bytes]     # index -> bytes
    merges: dict[tuple[int, int], int]  # index1,index2 -> new_index

def train_bpe(string: str, num_merges: int) -> BPETokenizerParams:  
    indices = list(map(int, string.encode("utf-8")))  
    merges: dict[tuple[int, int], int] = {}  # index1, index2 => merged index
    vocab: dict[int, bytes] = {x: bytes([x]) for x in range(256)}  # index -> bytes
    for i in range(num_merges):
        #Count the number of occurrences of each pair of tokens
        counts = defaultdict(int)
        for index1, index2 in zip(indices, indices[1:]):  # For each adjacent pair
            counts[(index1, index2)] += 1  # @inspect counts
        #Find the most common pair.
        pair = max(counts, key=counts.get) 
        index1, index2 = pair
        #Merge that pair.
        new_index = 256 + i 
        merges[pair] = new_index
        vocab[new_index] = vocab[index1] + vocab[index2]  
        indices = merge(indices, pair, new_index)  
    return BPETokenizerParams(vocab=vocab, merges=merges)
